## Function Calling Notebook

This notebook walks through the steps to run function calling through OpenAI. Function Calling mimics what Langchain does internally with tools and agents. The main difference is that Langchain executes the tool whereas OpenAI function calling comes back with the parameters to call the tool

In [106]:
from dotenv import load_dotenv
load_dotenv()

True

![Function Calling](images/function-calling.png)

In [98]:
import json
import openai
import os
from openai import OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

from langchain_community.tools.tavily_search import TavilySearchResults, TavilyAnswer

from transformers import pipeline
from termcolor import colored



#### Setting OpenAI Calling from Python

In [99]:
class Chatbot:
    def __init__(self):
        ## reading config file
        self.client = OpenAI()
        self.GPT_MODEL = "gpt-4-turbo-preview"

    def call_openai(self, query, tools=None):
        ## Add User Query to Messages
        if tools:
            completion = self.client.chat.completions.create(
                    model=self.GPT_MODEL,
                    messages=query,
                    response_format={"type": "json_object"},
                    temperature=0.2,
                    tools = tools,
                    tool_choice = "auto"
                    )
        else:
            completion = self.client.chat.completions.create(
                    model=self.GPT_MODEL,
                    messages=query,
                    response_format={"type": "json_object"},
                    temperature=0.4,
                    )
        
        ## Lets look at the output we got
        # print("Output from the model: ", completion.choices[0].message)
        # print("\n")
        tools_response = []

        ## Extract any content
        result = completion.choices[0].message.content
        ## If content is present, load it in JSON
        if result is not None:
            result = json.loads(result)
            result = result['response']

        ## Extract any tool calls
        tools_output =  completion.choices[0].message.tool_calls
        ## If tool_calls is in result iterate and extract all of them
        if tools_output is not None:
            for tool in tools_output:
                tools_response.append((tool.function.name, tool.function.arguments))
        return result, tools_response

In [100]:
query = "When does spring start in North Amercia?"
messages = []
messages.append({"role": "system", "content":"You are a friendly chatbot who likes to chat with users and extract relevant information. You respond back in JSON format. Put your answer in the key response"})
messages.append({"role": "user", "content": query})


chatbot = Chatbot()
result, tools_output = chatbot.call_openai(messages)
print(result, tools_output)

Output from the model:  ChatCompletionMessage(content='{\n  "response": "Spring officially starts with the vernal equinox, which usually occurs on March 20th or 21st in North America."\n}', role='assistant', function_call=None, tool_calls=None)


Spring officially starts with the vernal equinox, which usually occurs on March 20th or 21st in North America. []


### Define Functions using the Schema from OpenAI

We are going to have 3 functions
* Search - Tavily Search for the user query
* Calculator - Function that would do the math calculation
* Sentiment - Get text sentiment using transformer pipeline

In [101]:
tools = [
    {
      "type": "function",
      "function": {
        "name": "get_sentiment_text",
        "description": "Get the sentiment of the input text",
        "parameters": {
          "type": "object",
          "properties": {
            "query": {
              "type": "string",
              "description": "The text to analyze"
            },
          },
          "required": ["query"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_search_results",
        "description": "Search for results on a given topic",
        "parameters": {
          "type": "object",
          "properties": {
            "query": {
              "type": "string",
              "description": "The search query"
            },
            "limit": {
              "type": "integer",
              "description": "The number of results to return"
            }
          },
          "required": ["query"]
        }
      }
    },
   {
      "type": "function",
      "function": {
        "name": "calculator",
        "description": "Calculate the input query. Useful when you have a math computation",
        "parameters": {
          "type": "object",
          "properties": {
            "query": {
              "type": "string",
              "description": "The calculation to be done"
            }
        }
      }
      }
   }
  ]

In [102]:
query = "What is the product of 15 and 6"

messages = []
messages.append({"role": "system", "content":"You are a friendly chatbot who likes to chat with users and extract relevant information. You respond back in JSON format. Put your answer in the key response"})
messages.append({"role": "user", "content": query})


chatbot = Chatbot()
result, tools_output = chatbot.call_openai(messages, tools)
print(result, tools_output)


Output from the model:  ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_R4Oj0oUrPA4vszkbgWyzxZqL', function=Function(arguments='{"query":"15*6"}', name='calculator'), type='function')])


None [('calculator', '{"query":"15*6"}')]


### Write the defintion of functions

In [92]:
import re

def safe_calculator(query):
    """
    A simple calculator that evaluates basic arithmetic expressions from a string.
    Supports addition (+), subtraction (-), multiplication (*), and division (/).

    Parameters:
    - query: str, a mathematical expression as a string (e.g., '15*6')

    Returns:
    - The result of the arithmetic operation or an error message if the query is invalid.
    """
    # Pattern to match a basic arithmetic expression
    pattern = r'^(\d+(\.\d+)?)\s*([\+\-\*/])\s*(\d+(\.\d+)?)$'
    match = re.match(pattern, query)
    if not match:
        return "Error: Invalid input format."

    # Extract operands and operator
    a, operator, b = float(match.group(1)), match.group(3), float(match.group(4))

    # Perform calculation
    if operator == '+':
        return a + b
    elif operator == '-':
        return a - b
    elif operator == '*':
        return a * b
    elif operator == '/':
        if b == 0:
            return "Error: Division by zero is not allowed."
        return a / b
    else:
        return "Error: Unsupported operation."


def tavily_search(query, limit=2):
    """Function to execute Google Search."""
    try:
       tool = TavilyAnswer(max_results=limit)
       results = tool.invoke({"query":query})
    #    results = " ".join(item['content'] for item in search_results)
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def math_calculator(query):
    """Function to run Math Calculations."""
    try:
        results = safe_calculator(query)
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def text_sentiment(query):
    """Function to get sentiment of a text."""
    try:
        pipe = pipeline("sentiment-analysis")
        results = pipe(query)
    except Exception as e:
        results = f"query failed with error: {e}"
    return results


def function_executor(tools_response_LLM):
    """Tie above functions together so either can be executed"""
    name, params= tools_response_LLM[0]
    params = json.loads(params)
    if name == 'get_search_results':
        query = params['query']
        limit  = params.get('limit', 2)
        results = tavily_search(query, limit)
    elif name == 'calculator':
        query = params['query']
        results = math_calculator(query)
    elif name == 'get_sentiment_text':
        query = params['query']
        results = text_sentiment(query)
    else:
        results = f"Error: function {name} does not exist"
    return results

In [103]:
math_calculator("21/5")

4.2

In [104]:
text_sentiment("I hated the Dune 2 movie")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9995974898338318}]

In [105]:
tavily_search("How many Oscars has Robert Downey Jr. won?")

'Robert Downey Jr. has won one Oscar. He won the Academy Award for Best Actor in a Supporting Role at the 96th Academy Awards.'

### First Call to OpenAI

We will test the code by passing in the function arguments and making a first call to OpenAI

In [96]:
# user_request = """
# Find Harry Styles' age. What is their current age, multiplied by 2.1 ?
# """

# user_request = """
# What is current US unemployment rate? What will it be if it doubles?
# """

user_request = """
What are the latest reviews for Dune 2 movie? What is the sentiment of that
"""


system_message = '''
You are a friendly chatbot who looks at the tools they have and selects the best giving one.
You respond back in JSON format.
Put your response in the key response.
Important: Choose only one tool at a time. 
'''

messages = []
messages.append({"role": "system", "content":system_message})
messages.append({"role": "user", "content": user_request})
chatbot = Chatbot()

keep_running = True
while keep_running:
    print('/n')
    print(colored(messages, 'yellow'))
    result, tools_output = chatbot.call_openai(messages, tools=tools)

    if result is not None:
        messages.append({"role": "assistant", "content": result})
        print(colored(result, 'red'))
        keep_running = False

    if len(tools_output) >0:
        ## Run the tool request from LLM
        print(colored(tools_output, 'green'))
        function_results = function_executor(tools_output)
        print(colored(function_results, 'blue'))

        ### Pass the tool output back to LLM
        messages.append({"role": "function", "name": str(tools_output[0][0]), "content": str(function_results)})
        keep_running = True

/n
[{'role': 'system', 'content': '\nYou are a friendly chatbot who looks at the tools they have and selects the best giving one.\nYou respond back in JSON format.\nPut your response in the key response.\nImportant: Choose only one tool at a time. \n'}, {'role': 'user', 'content': '\nWhat are the latest reviews for Dune 2 movie? What is the sentiment of that\n'}]
[('get_search_results', '{"query": "Dune 2 movie reviews", "limit": 1}'), ('get_sentiment_text', '{"query": "Dune 2 movie reviews"}')]
Based on the information from the provided sources, "Dune: Part Two" has received mixed reviews. The movie is praised for its character development, particularly focusing on Paul Atreides' transformation into a potential leader. The film is likened to "The Lord of the Rings: The Two Towers" for building on established characters and intensifying the sense of continuous battle and danger. Additionally, Austin Butler's portrayal of Feyd-Rautha is highlighted for its intensity and balance against 

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[('get_sentiment_text', '{"query":"Based on the information from the provided sources, \\"Dune: Part Two\\" has received mixed reviews. The movie is praised for its character development, particularly focusing on Paul Atreides\' transformation into a potential leader. The film is likened to \\"The Lord of the Rings: The Two Towers\\" for building on established characters and intensifying the sense of continuous battle and danger. Additionally, Austin Butler\'s portrayal of Feyd-Rautha is highlighted for its intensity and balance against Timothée Chalamet\'s performance.\\n\\nOn the other hand, criticisms include the lack of new insights into religion as a form of control and the risks faced by indigenous communities. Despite this, the emotional depth and dynamics between characters are acknowledged as engaging elements of the film."}')]
[{'label': 'POSITIVE', 'score': 0.9950783252716064}]
/n
[{'role': 'system', 'content': '\nYou are a friendly chatbot who looks at the tools they have 

In [97]:

user_request = """
What is current US unemployment rate? What will it be if it doubles?
"""


system_message = '''
You are a friendly chatbot who looks at the tools they have and selects the best giving one.
You respond back in JSON format.
Put your response in the key response.
Important: Choose only one tool at a time. 
'''

messages = []
messages.append({"role": "system", "content":system_message})
messages.append({"role": "user", "content": user_request})
chatbot = Chatbot()

keep_running = True
while keep_running:
    print('/n')
    print(colored(messages, 'yellow'))
    result, tools_output = chatbot.call_openai(messages, tools=tools)

    if result is not None:
        messages.append({"role": "assistant", "content": result})
        print(colored(result, 'red'))
        keep_running = False

    if len(tools_output) >0:
        ## Run the tool request from LLM
        print(colored(tools_output, 'green'))
        function_results = function_executor(tools_output)
        print(colored(function_results, 'blue'))

        ### Pass the tool output back to LLM
        messages.append({"role": "function", "name": str(tools_output[0][0]), "content": str(function_results)})
        keep_running = True

/n
[{'role': 'system', 'content': '\nYou are a friendly chatbot who looks at the tools they have and selects the best giving one.\nYou respond back in JSON format.\nPut your response in the key response.\nImportant: Choose only one tool at a time. \n'}, {'role': 'user', 'content': '\nWhat is current US unemployment rate? What will it be if it doubles?\n'}]
[('get_search_results', '{"query":"current US unemployment rate","limit":1}')]
The current US unemployment rate is 3.7% as of November 2023. This rate marks a decrease from the previous month's rate of 3.9%.
/n
[{'role': 'system', 'content': '\nYou are a friendly chatbot who looks at the tools they have and selects the best giving one.\nYou respond back in JSON format.\nPut your response in the key response.\nImportant: Choose only one tool at a time. \n'}, {'role': 'user', 'content': '\nWhat is current US unemployment rate? What will it be if it doubles?\n'}, {'role': 'function', 'name': 'get_search_results', 'content': "The current